In [11]:
import numpy as np
import astropy.units as u
import astropy.constants as con
import emcee
import corner
import speclite as speclite; from speclite import filters
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib import cm
from matplotlib.artist import Artist
from chromatic import *
from chromatic import get_phoenix_photons
from scipy.optimize import minimize
from scipy.optimize import curve_fit


params = {'legend.fontsize': 'x-large',
          'figure.figsize': (12, 7),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)
plt.style.use('seaborn-deep')

In [12]:
all_orders = [53,54,55,56,57,58,59,
              60,61,62,63,64,65,66,67,68,69,
              70,71,72,73,74,75,76,77,78,79,
              80,81,82,83,84,85,86,87,88]

In [13]:
def quick_spec_chisq(parameters, extras, **kwargs):
    
    Tspec = parameters
    rwave, rflux, runc = extras
    
    if Tspec < 2300.:
        Tspec = 2300.
    if Tspec > 10000.:
        Tspec = 10000.
    
    _model = get_phoenix_photons(temperature=float(Tspec), wavelength = rwave,logg=4.4, metallicity=0.0)[1]
    model = _model/np.median(_model)
    chisq = np.nansum((rflux-model)**2/(runc)**2)

    return chisq

In [14]:
def sigma_clip_rainbow(rainbow,
                       order = None,
                       visit = 'define your visit in the photometry model!',
                       nsigma=10,**kwargs):
    
    #define some of the arrays
    r = rainbow
    preclip_r = r
    preclipped_data = r.flux
    preclipped_err = r.uncertainty
    function_to_minimize = quick_spec_chisq
    spectrum_number = ['first','second','third','fourth','fifth','sixth','seventh','eighth']
    Teffs = [None]*len(r.time)

    #loop through each time point
    for i in tqdm(range(len(r.time))):
        
        # Optimize a fit on the unprocessed data
        extras = [preclip_r.wavelength,preclipped_data[:,i],preclipped_err[:,i]]
        results = minimize(function_to_minimize, [3700.0], extras)
        initial_Tspec = results['x']
        if initial_Tspec < 2300.:
            initial_Tspec = 2300.
        if initial_Tspec > 10000.:
            initial_Tspec = 10000.
        
        # Calculate a model from the optimized _Tspec
        _m = get_phoenix_photons(temperature=float(initial_Tspec),wavelength=preclip_r.wavelength,logg=4.4,metallicity=0.0)[1]
        preclip_model = _m/np.median(_m)
        chisq = np.nansum((preclipped_data[:,i]-preclip_model)**2/(preclipped_err[:,i])**2)
        preclip_reduced_chisq = chisq/(len(preclip_r.wavelength)-1)
        err_kludge = np.sqrt(preclip_reduced_chisq)
        
        # Clip the emission lines, defined as points >2-sigma above the optimized model w/kludged errors
        # smooth to find where we're near an emission line
        N_boxcar_smooth = 6
        not_emission_line = ( (preclipped_data[:,i]-preclip_model)/(preclipped_err[:,i]*err_kludge) < 2.0 )
        near_emission_line = np.convolve(not_emission_line == False,
                                         np.ones(N_boxcar_smooth)/N_boxcar_smooth,
                                         mode='same')

        # only include points that are not near emission lines
        points_to_keep = near_emission_line == False
        preclip_r.wavelike["ok"] = points_to_keep
        clipped = preclip_r.trim(just_edges=False)
    
        if order == 53:
            _r = clipped
            a = _r.wavelength.value > 0.8823
            b = _r.wavelength.value < 0.8802
            _r.wavelike["ok"] = a+b
            clipped = _r.trim(just_edges=False)
        if order == 54:
            _r = clipped
            a = _r.wavelength.value > 0.8682
            b = _r.wavelength.value < 0.864
            _r.wavelike["ok"] = a+b
            clipped = _r.trim(just_edges=False)
        if order == 54:
            _r = clipped
            _r.wavelike["ok"] = _r.wavelength.value > 0.8585
            clipped = _r.trim(just_edges=False)
        if order == 55:
            _r = clipped
            a = _r.wavelength.value > 0.8435
            b = _r.wavelength.value < 0.854
            _r.wavelike["ok"] = a+b
            clipped = _r.trim(just_edges=False)
        if order == 56:
            _r = clipped
            a = _r.wavelength.value > 0.8326
            _r.wavelike["ok"] = a
            clipped = _r.trim(just_edges=False)
        if order == 58:
            _r = clipped
            a = _r.wavelength.value > 0.802
            _r.wavelike["ok"] = a
            clipped = _r.trim(just_edges=False)
        if order == 59:
            _r = clipped
            a = _r.wavelength.value > 0.786
            b = _r.wavelength.value < 0.797
            _r.wavelike["ok"] = a+b
            clipped = _r.trim(just_edges=False)
        if order == 60:
            _r = clipped
            a = _r.wavelength.value > 0.773
            b = _r.wavelength.value < 0.784
            _r.wavelike["ok"] = a+b
            clipped = _r.trim(just_edges=False)
        if order == 62:
            _r = clipped
            a = _r.wavelength.value > 0.752
            b = _r.wavelength.value < 0.750
            _r.wavelike["ok"] = a+b
            clipped = _r.trim(just_edges=False)
        if order == 63:
            _r = clipped
            a = _r.wavelength.value > 0.7355
            b = _r.wavelength.value < 0.746
            _r.wavelike["ok"] = a+b
            clipped = _r.trim(just_edges=False)    
        if order == 69:
            _r = clipped
            b = _r.wavelength.value < 0.681
            _r.wavelike["ok"] = b
            clipped = _r.trim(just_edges=False)    
        if order == 71:
            _r = clipped
            a = _r.wavelength.value > 0.653
            b = _r.wavelength.value < 0.662
            _r.wavelike["ok"] = a+b
            clipped = _r.trim(just_edges=False)    
        if order == 72:
            _r = clipped
            b = _r.wavelength.value < 0.6525
            _r.wavelike["ok"] = b
            clipped = _r.trim(just_edges=False)
        if order == 73:
            _r = clipped
            a = _r.wavelength.value > 0.646
            b = _r.wavelength.value < 0.6375
            _r.wavelike["ok"] = a+b
            clipped = _r.trim(just_edges=False)
        if order == 75:
            _r = clipped
            a = _r.wavelength.value > 0.6196
            b = _r.wavelength.value < 0.6265
            _r.wavelike["ok"] = a+b
            clipped = _r.trim(just_edges=False)
        if order == 76:
            _r = clipped
            a = _r.wavelength.value > 0.6095
            b = _r.wavelength.value < 0.6185
            _r.wavelike["ok"] = a+b
            clipped = _r.trim(just_edges=False)    
        if order == 77:
            _r = clipped
            a = _r.wavelength.value < 0.61
            _r.wavelike["ok"] = a
            clipped = _r.trim(just_edges=False)   
        if order == 81:
            _r = clipped
            a = _r.wavelength.value > 0.5725
            b = _r.wavelength.value < 0.580
            _r.wavelike["ok"] = a+b
            clipped = _r.trim(just_edges=False)
        if order == 82:
            _r = clipped
            a = _r.wavelength.value > 0.565
            b = _r.wavelength.value < 0.572
            _r.wavelike["ok"] = a+b
            clipped = _r.trim(just_edges=False)
        if order == 86:
            _r = clipped
            _r.wavelike["ok"] = _r.wavelength.value < 0.5445
            clipped = _r.trim(just_edges=False)       
        # Do that again, removing nsigma points from above or below the model
        extras = [clipped.wavelength, clipped.flux[:,i], clipped.uncertainty[:,i]]
        results = minimize(function_to_minimize, [3700.0], extras)
        _Tspec = results['x']
        if _Tspec < 2300.:
            _Tspec = 2300.
        if _Tspec > 10000.:
            _Tspec = 10000.
        _m = get_phoenix_photons(temperature=float(_Tspec),wavelength=clipped.wavelength,logg=4.4,metallicity=0.0)[1]
        model = _m/np.median(_m)
        chisq = np.nansum((clipped.flux[:,i]-model)**2/(clipped.uncertainty[:,i])**2)
        reduced_chisq = chisq/(len(clipped.wavelength)-1)
        err_kludge = np.sqrt(reduced_chisq)
        
        points_to_keep = ( np.abs(clipped.flux[:,i]-model)/(clipped.uncertainty[:,i]*err_kludge) <= nsigma )
        clipped.wavelike["ok"] = points_to_keep
        reclipped = clipped.trim(just_edges=False)
        
        # Once more, removing errors > 20%
        extras = [reclipped.wavelength,reclipped.flux[:,i],reclipped.uncertainty[:,i]]
        results = minimize(function_to_minimize, [3700], extras)
        _Tspec = results['x']
        if _Tspec < 2300.:
            _Tspec = 2300.
        if _Tspec > 10000.:
            _Tspec = 10000.
        _m = get_phoenix_photons(temperature=float(_Tspec),wavelength=reclipped.wavelength,logg=4.4,metallicity=0.0)[1]
        model = _m/np.median(_m)
        chisq = np.nansum((reclipped.flux[:,i]-model)**2/(reclipped.uncertainty[:,i])**2)
        reduced_chisq = chisq/(len(reclipped.wavelength)-1)
        err_kludge = np.sqrt(reduced_chisq)
        
        err_outliers = ( clipped.uncertainty[:,i]*err_kludge < 0.2 )
        clipped.wavelike["ok"] = err_outliers
        final = clipped.trim(just_edges=False)
        
        # Calculate a final error adjustment
        extras = [final.wavelength,final.flux[:,i],final.uncertainty[:,i]]
        results = minimize(function_to_minimize, [3700.], extras)
        Teff = results['x']
        if Teff < 2300.:
            Teff = 2300.
        if Teff > 10000.:
            Teff = 10000.
        _m = get_phoenix_photons(temperature=float(Teff),wavelength=final.wavelength,logg=4.4,metallicity=0.0)[1]
        finalmodel = _m/np.median(_m)
        chisq = np.nansum((final.flux[:,i]-finalmodel)**2/(final.uncertainty[:,i])**2)
        finreduced_chisq = chisq/(len(final.wavelength)-1)
        finerr_kludge = np.sqrt(finreduced_chisq)
        processed_rainbow = final
        processed_rainbow.uncertainty = final.uncertainty*finerr_kludge
        Teffs[i] = int(Teff)         
        sanity_check = np.nansum((final.flux[:,i]-finalmodel)**2/(final.uncertainty[:,i])**2)
        should_be_1 = sanity_check/(len(final.wavelength)-1)
      
    return processed_rainbow

In [15]:
def initialize(order = None,
               visit = 'Initialization visit is not defined!',
               dw=0.05*u.nm):
    """
    Load in and process NRES spectra
    """
    
    #define visit-specific parameters (why not use dicts? good question)
    if visit == 'fall':
        phot_amplitudes = np.array([0.07,0.066,0.041])
        phot_errs = np.array([0.01,0.01,0.01]) 
        vshift= -9.9
        data_path = 'data/NRES/fall_spectra/*e92-1d.fits.fz'
        edges = np.array([2459451.0,2459452.0,2459454.0,2459457.0,
                          2459459.0,2459460.0,2459461.0])*u.day
        _ntimes = len(edges)
        
    if visit == 'spring':
        phot_amplitudes = np.array([0.07,0.071])
        phot_errs = np.array([0.004,0.004])
        vshift = 33.3
        data_path = 'data/NRES/spring_spectra/*e92-1d.fits.fz'
        edges = np.array([2459680.5,2459681.5,2459682.5,
                          2459683.5,2459684.5,2459685.5,
                          2459686.5,2459687.5,2459688.5])*u.day
        _ntimes = len(edges)
    
    #create the rainbow object and apply the velocity shift
    r = Rainbow(data_path, format='nres', order=order)
    shifted_r = r.align_wavelengths().shift(vshift * u.km/u.s)

    #correct for atmospheric transmission from skycalc outputs
    transmission_data = np.loadtxt('data/transmission_comp.dat')
    mol_wave = transmission_data[:,0]*(1e-3) * u.micron
    mol_data = transmission_data[:,1]
    this_orders_tellurics = bintogrid(mol_wave.value,mol_data,newx=shifted_r.wavelength.value,
                                      visualize=False)
    transparent_wavelengths = (this_orders_tellurics['y'] > 0.995)
    shifted_r.wavelike["ok"] = transparent_wavelengths

    #bin the rainbow so the clipping doesn't take a thousand years
    preclipped_rainbow = shifted_r.trim(just_edges=False).bin(dw=dw,
                                                              time_edges=edges,
                                                              ntimes=_ntimes,
                                                              minimum_points_per_bin=1)
    
    print(f'Telluric lines contaminate {int( (len(r.wavelength)-len(preclipped_rainbow.wavelength))/len(r.wavelength) )}% of this spectrum.')

    #run the sigma clipping function to return a final processed rainbow
    processed_data = sigma_clip_rainbow(rainbow=preclipped_rainbow,order=order,visit=visit,nsigma=4.0)
    
    return processed_data,this_orders_tellurics,phot_amplitudes,phot_errs

In [16]:
def time_domain_spectrum_model(parameters=[0.20,0.06,3200.,3900.],
                               order = None,
                               visit = 'define your visit in the time domain spectrum model!',
                               spot_spec = 0,
                               amb_spec = 0,
                               plot=False,**kwargs):
    if visit == 'fall':
        phase = 1.88
        T0 = (59455.48 + 2400000.5)
    if visit == 'spring':
        phase = 2.0
        T0 = (59455.48 + 2400000.5) + (27.*8.46)
    T_rot = 4.863 # AU mic rotation period, days
    f_spot,df_spot,T_spot, T_amb = parameters
    
    # Calculate the average spectrum model
    combined = f_spot*spot_spec + (1.-f_spot)*amb_spec
    # ft_spot = f_spot + df_spot*np.sin(2*np.pi*data_times.value/T_rot + phase)
    # combined = ft_spot*spot_spec + (1.-ft_spot)*amb_spec
    spec_model = combined/np.median(combined)
    
#     # Calculate dS/S
#     contrast = (1.-spot_spec/amb_spec)
#     dS_over_S = np.abs(-df_spot * ( contrast / (1.-f_spot*(contrast)) ))
    
#     # # Set up and calculate the sinusoidal S(lambda,t) model
#     spec_model = np.ones_like(nres_rainbow.flux)
#     nres_spec_comparison = np.ones_like(nres_rainbow.flux)
#     for i in range(len(data_times)):
#         S_t_model = 0.0
#         S_t_model = S_avg_model * ( 1. + dS_over_S * np.cos(2.0*np.pi*data_times[i].value/T_rot + phase) )
#         spec_model[:,i] = S_t_model/np.median(S_t_model)
#         nres_spec_comparison[:,i] = nres_rainbow.flux[:,i]/nres_avg_1dspec

#     # Set up and calculate the sinusoidal S(lambda,t) model
#     spec_model = np.ones_like(nres_rainbow.flux)
#     nres_spec_comparison = np.ones_like(nres_rainbow.flux)
#     for i in range(len(data_times)):
#         S_t_model = 0.0
#         ft_spot = f_spot - df_spot*np.sin(2*np.pi*data_times[i].value/T_rot + phase)
#         combined = ft_spot*spot_spec + (1.-ft_spot)*amb_spec
#         S_t_model = combined/np.median(combined)
#         spec_model[:,i] = S_t_model
#         nres_spec_comparison[:,i] = nres_rainbow.flux[:,i]/nres_avg_1dspec
        
    chisq = np.nansum((nres_rainbow.flux - spec_model)**2/(nres_rainbow.uncertainty)**2)
    reduced_chisq = chisq/(len(data_wave)*len(data_times)-4)
    kludge = np.sqrt(reduced_chisq)

    if plot:

        model_data_times = np.linspace(data_times[0].value-1,data_times[-1].value+1,300)
        cmap = cm.get_cmap('cividis', len(data_times))
        colormap = cmap(np.linspace(0, 1, len(data_times)))

        fig, axs = plt.subplots(2,3,figsize=(18,10))
        title_label=f'Order {order} | fspot+/-'+r'$\Delta$'+f'fspot = {f_spot:.2f}+/-{df_spot:.2f} | Tspot: {int(T_spot/10)*10} | Tamb: {int(T_amb/10)*10} | err kludge: {kludge:0.1f} | reduced chisq: {reduced_chisq:.2f}'
        fig.suptitle(title_label,fontsize=18)

        ax1 = axs[0,0]
        ax2 = axs[1,0]
        ax3 = axs[0,1]
        ax4 = axs[1,1]
        ax5 = axs[0,2]
        ax6 = axs[1,2]

        for i in range(len(data_times)):
            ax3.errorbar(data_wave,nres_rainbow.flux[:,i],yerr=nres_rainbow.uncertainty[:,i],alpha=0.6,
                         color=colormap[i],fmt='')
            ax4.plot(data_wave,(spec_model[:,i]-nres_rainbow.flux[:,i])/nres_rainbow.uncertainty[:,i],
                     alpha=0.6,color=colormap[i])
        
        ax1.set_title(r'Model dS/dT',fontsize=18)
        ax1.plot(data_wave,(spot_spec/np.median(spot_spec)-amb_spec/np.median(amb_spec))/(T_amb-T_spot),color='k')
        ax1.set_xlim(this_orders_tellurics['x'][0],this_orders_tellurics['x'][-1])
        ax1.set_ylim(0.000001,0.1)
        ax1.set_ylabel('dS/dT',fontsize=18)
        ax1.set_yscale('log')
        
        ax2.set_title(r'Model $\Delta$S/S',fontsize=18)
        ax2.plot(data_wave,np.abs(dS_over_S))
        ax2.set_xlim(this_orders_tellurics['x'][0],this_orders_tellurics['x'][-1])
        ax2.set_xlabel(r'Wavelength ($\mu$m)',fontsize=18)
        ax2.set_ylim(0,0.1)
        ax2.set_ylabel(r'$\Delta$S/S',fontsize=18)        

        ax3.set_title('NRES data in time',fontsize=18)
        ax3.plot(data_wave, np.median(spec_model,axis=1),color='k',zorder=10,label='median model spectrum')
        ax3.set_xlim(this_orders_tellurics['x'][0],this_orders_tellurics['x'][-1])
        ax3.set_ylim(0.5,1.5)
        ax3.set_ylabel('Relative flux',fontsize=18)
        ax3.legend()
           
        ax4.set_title(r'Residuals',fontsize=18)
        ax4.set_xlim(this_orders_tellurics['x'][0],this_orders_tellurics['x'][-1])
        ax4.set_xlabel(r'Wavelength ($\mu$m)',fontsize=18)
        ax4.set_ylabel(r'$\sigma$',fontsize=18)

        ax5.set_title(r'NRES S($\lambda$,t)/S($\lambda$)',fontsize=18)
        speccompplot = ax5.imshow(np.transpose(nres_spec_comparison),aspect='auto',cmap='Reds',
                                  extent=[data_wave[0].value,data_wave[-1].value,data_times[-1].value,data_times[0].value])
        ax5.set_xlabel(r'Wavelength ($\mu$m)',fontsize=18)
        ax5.set_ylabel('BJD',fontsize=18)
        plt.colorbar(speccompplot, ax = ax5)
        
        ax6.set_title('PHOENIX Model Spectra',fontsize=18)
        specplot= ax6.imshow(np.transpose(spec_model),aspect='auto',cmap='Reds',
                             extent=[data_wave[0].value,data_wave[-1].value,data_times[-1].value,data_times[0].value])
        ax6.set_ylabel('BJD',fontsize=18)
        ax6.set_xlabel(r'Wavelength ($\mu$m)',fontsize=18)
        plt.colorbar(specplot, ax = ax6) 

        plt.savefig(f'figs/Timeseries_models/{visit}/Order_{order}/{visit}_{order}_Timeseries_{file_label}.png',dpi=300)
        plt.show()

    return spec_model,chisq

In [17]:
orders_to_model = [53,54,55,56,57,58,59,
                   60,61,62,63,64,65,66,67,68,69,
                   70,71,72,73,74,75,76,77,78,79,
                   80,81,82,83,84,85]

In [18]:
# for order in tqdm(orders_to_model):
order = 53
visit = 'fall'
file_label = 'timeseries_testing'
dw = 0.05 * u.nm
nsteps = 10
do_Photometry=False
do_Teff = False
do_AvgSpec=False
do_TimeSpec=False

phase =  1.9# + np.pi/2.

# Begin initializing things

#     print('beginning MCMC..')
#     do_mcmc(order = order,
#             visit=visit,
#             nsteps = nsteps,
#             plot=True)

In [19]:
for visit in ['fall','spring']:
    for order in tqdm(orders_to_model):
        nres_rainbow,this_orders_tellurics,phot_amplitudes,phot_errs = initialize(order=order,
                                                                                  visit=visit,
                                                                                  dw=0.01*u.nm)
        data_wave = nres_rainbow.wavelength
        data_times = nres_rainbow.timelike['time']
        nres_avg_1dspec = nres_rainbow.get_average_spectrum()
        fig, ax = plt.subplots(2,1,figsize=(6,7))
        corrcoeffs = [0]*len(data_wave)
        
        cmap = cm.get_cmap('cividis', len(data_wave))
        colormap = cmap(np.linspace(0, 1, len(data_wave)))

        if visit=='fall':
            phase = 1.88
        if visit == 'spring':
            phase = 2.02
        for i in range(len(data_wave)):
            if i == 0:
                i += 1
            x_array = np.array([None]*len(data_times))
            ratio_array = np.array([None]*len(data_times))
            for time in range(len(data_times)):
                x_array[time] = np.cos(2.0*np.pi*data_times[time].value/4.863 + phase)
                ratio_array[time] = nres_rainbow[i,time].flux[0]/nres_avg_1dspec[i]
            ax[0].scatter(x_array,ratio_array,color=colormap[i],alpha=0.1,edgecolor='none')
            ax[0].set_xlabel('Sine term')
            ax[0].set_ylabel(r'S($\lambda$,t)/S$_{\rm{avg}}$')
            ax[0].set_ylim(0.95,1.05)
            ax[0].set_title(f'Order {order} | phase: {phase:.2f}')
            corrcoeffs[i] = np.corrcoef(x_array, ratio_array,dtype=float)[0,1]
        ax[1].scatter(data_wave,corrcoeffs,alpha=0.5)
        ax[1].set_xlabel('wavelength')
        ax[1].set_ylabel(r'Correlation coefficient')
        ax[1].set_ylim(-1,1)

        plt.savefig(f'figs/diagnostics/{visit}{order}_p{int(phase*100)}_timeseries_testing.png',dpi=200)
        plt.clf()

  0%|          | 0/33 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/3466 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

...

<Figure size 1800x2100 with 0 Axes>

<Figure size 1800x2100 with 0 Axes>

<Figure size 1800x2100 with 0 Axes>

<Figure size 1800x2100 with 0 Axes>

<Figure size 1800x2100 with 0 Axes>